In [20]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
%autoreload

In [3]:
import xlrd
import plotly.express as px
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import plan_utils as utils
import plan_var as V
from plan_loader import *

In [23]:
#ПП 25.08.2020

In [4]:
data = PlanLoader('план.xlsx').df
data.head(15)

,Заливка,Заливка_1,Навеска,Навеска_1,Передача ЭМУ,Сборка щитов,Монтаж СН,Шина УВР/ТР-Р,"Сборка 0,4 кВ","Втор-ка 0,4 кВ",Сборка 6/10 кВ,Втор-ка 6/10 кВ,"Уст-ка 0,4/10кВ",Установ-ка RM-6,Наладка,Комплек-тация,ОТГРУЗКА,ОТГРУЗКА_1
1250,готов,готов,20.11,20-25.11,готов,20-25.11,20-23.11,готов,готов,27-29.11,нет,нет,19.11,20.11,05.12,06.12,2019-12-06,нет
1253,готов,готов,26.11,26-27.11,25.11,26-28.11,25-26.11,готов,готов,02-04.13,нет,нет,25.12,27.12,05.13,06.13,2019-12-10,нет
1259,нет,нет,нет,нет,нет,02-05.12,нет,нет,нет,нет,нет,нет,нет,нет,09.12,09.12,2019-12-10,нет
1255,готов,готов,28.11,28-30.11,26.11,05-07.12,27-29.11,20.11/22.11,25-27.11,05-09.12,нет,нет,29.11,03.12,09.12,10.12,2019-12-11,нет
1256,готов,готов,02.12,02-04.12,29.11,07-10.12,30-31.11,25.11/27.11,28.11-30.11,06-10.12,нет,нет,03.12,03.12,10.12,11.12,2019-12-12,нет
1251,готов,готов,05.12,05-07.12,02.12,07-10.12,02-04.12,28.11/02.12,02-04.12,06-10.12,нет,нет,05.12,03.12,11.12,11.12,2019-12-12,нет
1260,нет,нет,нет,нет,нет,07-10.12,нет,нет,нет,нет,нет,нет,нет,нет,11.12,12.12,2019-12-13,нет
1257,готов,готов,09.12,09-11.12,04.12,05-10.12,05-07.12,03.12/05.12,05-07.12,10-13.12,нет,нет,05.12,03.12,13.12,13.12,2019-12-16,нет
1258,нет,нет,нет,нет,нет,нет,нет,нет,нет,нет,нет,нет,нет,11.12,16.12,17.12,2019-12-18,нет
1264,готов,готов,готов,готов,готов,10-12.12,09.12,05.12/07.12,05-07.12,16-18.12,нет,нет,09.12,12.12,19.12,20.12,2019-12-23,нет


In [30]:
D = {7 : ['StShinaUvr', 'FnShinaUvr', 'StShinaTr', 'FnShinaTr'],
                 12: ['StInst04', 'FnInst04', 'StInst10', 'FnInst10']}
list(D.keys())[0]


7

In [39]:
%autoreload
df_names = V.get_FinalFinalColumnNames()
df_names

['StBuildBot',
 'FnBuildBot',
 'StBuildTop',
 'FnBuildTop',
 'StMetBot',
 'FnMetBot',
 'StMetTop',
 'FnMetTop',
 'StEmu',
 'FnEmu',
 'StShit',
 'FnShit',
 'StSn',
 'FnSn',
 'StShinaUvr',
 'FnShinaUvr',
 'StShinaTr',
 'FnShinaTr',
 'St04',
 'Fn04',
 'StSec04',
 'FnSec04',
 'St10',
 'Fn10',
 'StSec10',
 'FnSec10',
 'StInst04',
 'FnInst04',
 'StInst10',
 'FnInst10',
 'StRm',
 'FnRm',
 'StMain',
 'FnMain',
 'StCompl',
 'FnCompl',
 'Fin',
 'NewFin']

In [32]:
'27-31.08'.split('/')

['27-31.08']

In [41]:
%autoreload
d = df.apply(lambda row: utils.reformat_dataframe(row),axis=1)
d = d[V.get_FinalFinalColumnNames()]

ValueError: time data '23/23.09' does not match format '%d.%m'

In [34]:
d.shape

(11, 38)

In [35]:
dd = d.iloc[0:10]

data = pd.DataFrame()

for ser in dd.iloc:
    i = 0
    while i < len(ser) - 2:
        if type(ser[i]) == pd._libs.tslibs.timestamps.Timestamp:
            data = data.append(pd.DataFrame([dict(Task=ser.index[i][2:],
                                                Start=ser[i],
                                                Finish=ser[i+1] + timedelta(hours=23, minutes=59),
                                                zakaz=ser.name)]), ignore_index=True)
            i+=1
        i+=1

data

,Task,Start,Finish,zakaz
0,Sn,2020-08-31,2020-09-01 23:59:00,1330
1,ShinaTr,2020-08-26,2020-08-26 23:59:00,1330
2,04,2020-08-26,2020-08-28 23:59:00,1330
3,Sec04,2020-08-28,2020-08-31 23:59:00,1330
4,Inst10,2020-08-31,2020-08-31 23:59:00,1330
...,...,...,...,...
138,Sec10,2020-09-19,2020-10-10 23:59:00,1328
139,Inst04,2020-10-30,2020-10-30 23:59:00,1328
140,Inst10,2020-10-30,2020-10-30 23:59:00,1328
141,Main,2020-10-12,2020-10-12 23:59:00,1328


In [36]:
fig = px.timeline(data, x_start="Start", x_end="Finish", y="Task", color='zakaz')
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()